# Calibration Model 03: Generátor syntetických dat (Synthetic Data Generator)

### Počet kalibrovaných parametrů: 3

In [1]:
# Instalace potřebných knihoven
#%pip install pandas
#%pip install numpy
#%pip install seaborn matplotlib
#%pip install pymc
#%pip install arviz
#%pip install ipywidgets
#%pip install jupyterlab_widgets
#%pip install pytensor
#%pip install ipywidgets jupyterlab_widgets

In [2]:
# Import potřebných knihoven
import pandas as pd
import numpy as np

import pymc as pm
import arviz as az
import pytensor.tensor as pt

import seaborn as sns
import matplotlib.pyplot as plt

### Vstupní data

In [3]:
### Načtení z formátu netCDF

# Soubor je načten a přiřazen do proměnné ‚trace‘
other_path = '../../data/05_Calibration/posterior_trace_three.nc'
trace = az.from_netcdf(other_path)

In [4]:
az.summary(trace)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
v_load,0.494,0.317,0.173,0.935,0.153,0.001,6.0,100.0,1.73
v_unload,0.661,0.487,0.200,1.408,0.230,0.003,6.0,109.0,1.73
accel,0.205,0.007,0.200,0.217,0.001,0.001,42.0,131.0,1.06
sigma,3.405,0.095,3.230,3.584,0.001,0.001,9956.0,7339.0,1.00


### Definice a nastavení parametrů robotického systému

In [5]:
# SPECIFIKACE TECHNOLOGICKÉHO PROCESU ZDĚNÍ

time_suck_on = 1.5        # s, doba přisátí zdicího prvku
time_suck_off = 0.5       # s, doba odsátí (uvolnění) prvku
time_verif_refer = 3.5    # s, doba pohybu z místa ověření do referenčního bodu
time_refer_pallete = 2.1  # s, doba pohybu z referenčního bodu k paletě

time_pallete_verif = 9.8  # s, doba pohybu od palety k referenčnímu bodu

### Definice funkce pro výpočet celkové doby pracovního cyklu

In [ ]:
def simulate_time(dist, v_load, v_unload, accel):
    """
    Funkce pro výpočet celkové doby pracovního cyklu robotického zdění.

    Návratová hodnota:
    total_time (float): celková doba pracovního cyklu [s]
    """

    # výpočet času potřebného k dosažení maximální rychlosti
    time_to_max_load_speed = v_load / accel
    dist_to_max_load_speed = (1/2) * accel * time_to_max_load_speed  # uražená dráha při akceleraci

    # výpočet času potřebného k dosažení 0 rychlosti
    time_to_max_unload_speed = v_unload / accel
    dist_to_max_unload_speed = (1/2) * accel * time_to_max_unload_speed  # uražená dráha při deakceleraci


    # pevné technologické časy (manipulace a přesuny mezi pevnými body)
    total_time = time_suck_on + time_pallete_verif
    total_time += time_suck_off + time_suck_on + time_verif_refer

    # pohyb s naloženým prvkem (převod mm → m)
    total_time += (dist - dist_to_max_load_speed) / 1_000 / v_load

    # manipulace v cílové poloze a návrat
    total_time += time_suck_off + time_refer_pallete

    # pohyb bez zátěže (zpětný pohyb)
    total_time += (dist - dist_to_max_unload_speed) / 1_000 / v_unload

    # započtení akceleračních časů
    total_time += time_to_max_load_speed + time_to_max_unload_speed
    return total_time

In [7]:
dist_min = 1_000
dist_max = 10_000

dist_range = np.random.uniform(dist_min, dist_max, 1000)

### Generátor synteticých dat

In [ ]:
# =====================================================
# 1️ EXTRAKCE POSTERIOR VZORKŮ
# =====================================================

# Sloučení chain + draw do jedné dimenze
posterior = trace.posterior.stack(sample=("chain", "draw"))

# Extrakce parametrů
v_load_samples = posterior["v_load"].values
v_unload_samples = posterior["v_unload"].values
accel_samples = posterior["accel"].values
sigma_samples = posterior["sigma"].values

n_samples = len(v_load_samples)

# =====================================================
# 2️ MONTE CARLO SIMULACE
# =====================================================

simulated = []
for d in dist_range:

    T_samples_for_dist = []

    for i in range(n_samples):

        T_det = simulate_time(
            dist=d,
            v_load=v_load_samples[i],
            v_unload=v_unload_samples[i],
            accel=accel_samples[i]
        )

        noise = np.random.normal(0, sigma_samples[i], size=np.shape(T_det))

        T_sim = T_det + noise

        T_samples_for_dist.append(T_sim)

    simulated.append(T_samples_for_dist)

simulated_data = np.array(simulated)

Průměr: [58.22441468 58.58835231 58.53456593 ... 57.76388424 58.42108622
 58.32048016]
95% interval: [29.74436671 31.84852386 31.32158418 ... 29.69186742 29.45202642
 31.38012264] - [86.25494314 86.60331799 85.9044224  ... 85.81393026 86.85364555
 86.36247698]
Směrodatná odchylka: [16.86358536 16.51259433 16.50011256 ... 16.50355374 17.17652763
 16.53545302]


In [19]:
simulated_data.shape

(1000, 12000)

In [16]:
# =====================================================
# 3️ STATISTICKÉ VYHODNOCENÍ
# =====================================================

T_mean = simulated_data.mean(axis=1)
T_median = np.median(simulated_data, axis=1)
T_ci_lower = np.percentile(simulated_data, 2.5, axis=1)
T_ci_upper = np.percentile(simulated_data, 97.5, axis=1)
T_std = simulated_data.std(axis=1)

In [17]:
# =====================================================
# 4️ VÝSTUPNÍ SHRNUTÍ
# =====================================================

print("Průměr:", T_mean)
print("95% interval:", T_ci_lower, "-", T_ci_upper)
print("Směrodatná odchylka:", T_std)

Průměr: [86.44576776 59.15341687 67.85618026 70.15836003 34.36489544 69.81157055
 45.66255908 72.65847706 56.24152875 48.98043555 43.48650725 78.61822019
 39.76651887 33.22027147 61.22795225 61.27549723 62.62536611 44.63080846
 32.27572845 36.27131137 56.66745809 52.34550744 63.39303436 76.38483522
 61.4628168  51.69839274 74.6846072  48.17396079 67.41050092 51.01929733
 38.63705067 46.07694627 86.37442722 40.43043926 34.15216694 55.6893014
 50.17122794 39.94331563 60.46460805 79.77250918 59.52500872 77.56542461
 43.53053779 71.2292007  54.08723045 32.8451705  53.70035071 78.00177753
 49.17528202 62.52306049 40.37463812 55.98257774 43.0493438  71.27919578
 42.17997102 37.80123725 48.80142481 41.4828786  41.90612189 50.30669772
 64.14232811 73.5327048  80.88195227 31.24200654 37.32327285 84.42540552
 34.4977155  60.26137133 74.78481207 58.63296611 84.27400671 35.06339611
 65.16828245 54.13480351 39.16088536 74.27469425 60.30555188 67.58590928
 54.18543395 38.6954169  59.87322384 54.1974

### Export datové sady do formátu netCDF a CSV

https://www.unidata.ucar.edu/software/netcdf

In [ ]:
df_synt.to_csv("../../data/05_Calibration/syntetic_data.csv", index=False)

### Autor / Organizace / Datum

Vjačeslav Usmanov, ČVUT v Praze, Fakulta stavební

###### Přehled změn


|  Datum (YYYY-MM-DD) |  Verze | Autor změny  |  Popis změny |
|---|---|---|---|
| 2026-01-31 | 1.1 | Vjačeslav Usmanov| added CM_03_SyntGenerator.ipynb |
| 2026-02-18 | 1.2 | Vjačeslav Usmanov| changed CM_03_SyntGenerator.ipynb |